# Filtering Microcap Alts

In [1]:
#%% Importing modules and data
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import quandl as qd
import seaborn as sns
import matplotlib.dates as dates
import matplotlib.ticker as ticker
from pandas_datareader import data
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import os
import glob
from time import sleep


# Functions for pulling from Coinmarketcap
- coindfpull: pulls coin information from cmc including total coin supply
- tokendfpull: pulls token information from cmc including total token supply
- coincircdf: pulls coin circulating supply from cmc
- tokencircdf: pulls token circulating supply from cmc

In [2]:
def coindfpull(pagenum):
    site = 'https://coinmarketcap.com/coins/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[9].split('data-btc="')[1].split(' " data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[9].split('data-btc="')[1].split(' " data-usd="')[0]]
            pricesusd = pricesusd + [columns[15].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            pricesbtc = pricesbtc + [columns[15].split('" data-btc="')[1].split('" data-usd="')[0]]
            volumes24husd = volumes24husd + [columns[18].split('" data-btc="')[1].split('" data-usd="')[1].split('" ')[0]]
            volumes24hbtc = volumes24hbtc + [columns[18].split('" data-btc="')[1].split('" data-usd="')[0]]
            totalsupplies = totalsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def tokendfpull(pagenum):
    site = 'https://coinmarketcap.com/tokens/views/market-cap-by-total-supply/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 

    ranks = []
    currnames = []
    platforms = []
    mcapsusd = []
    mcapsbtc = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    volumes24hbtc = []
    totalsupplies = []
    #changes24husd = []
    #changes24hbtc = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            ranks = ranks + [columns[2].replace(' ','')]
            platforms = platforms + [columns[0].split('data-platformsymbol="')[1].split('" ')[0]]
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            mcapsusd = mcapsusd + [columns[10].split('data-btc="')[1].split(' " ')[1].split('data-usd="')[1].replace('">','')]
            mcapsbtc = mcapsbtc + [columns[10].split('data-btc="')[1].split(' " ')[0]]
            pricesusd = pricesusd + [columns[16].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            pricesbtc = pricesbtc + [columns[16].split('data-btc="')[1].split('" ')[0]]
            volumes24husd = volumes24husd + [columns[19].split('data-btc="')[1].split('" ')[2].replace('data-usd="','')]
            volumes24hbtc = volumes24hbtc + [columns[19].split('data-btc="')[1].split('" ')[0]]
            totalsupplies = totalsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
            #changes24husd = changes24husd + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[1].split('">')[0]]
            #changes24hbtc = changes24hbtc + [columns[-3].split('" data-btc="')[1].split('" data-usd="')[0].split('" ')[0]]
    
    crypto_dict = {"Rank": ranks,
                   "Names": currnames,
                   "Platform": platforms,
                   "MktCapUSD": mcapsusd,
                   "MktCapBTC": mcapsbtc,
                   "PricesUSD": pricesusd,
                   "PricesBTC": pricesbtc,
                   "Vol24USD": volumes24husd,
                   "Vol24BTC": volumes24hbtc,
                   "TotalSupply": totalsupplies}
                   #"Change24USD": changes24husd,
                   #"Change24BTC": changes24hbtc
                  #}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    
    return outdf

def coincircdf(pagenum):
    site = 'https://coinmarketcap.com/coins/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[21].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

def tokencirdf(pagenum):
    site = 'https://coinmarketcap.com/tokens/' + str(pagenum)
    res = requests.get(site)
    if str(res) == "<Response [404]>":
        return "404 Response"
    
    soup = bs(res.text, 'lxml')

    table = soup.find_all('table')[0] 
    
    currnames = []
    circulatingsupplies = []

    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if '#' in columns[1]:
            continue
        else:
            currnames = currnames + [columns[6].split('<span class="currency-symbol"><a href="/currencies/')[1].split('/')[0]]
            circulatingsupplies = circulatingsupplies + [columns[22].split('data-supply="')[1].split('" ')[0].replace('">','')]
    
    crypto_dict = {"Names": currnames,
                   "CirculatingSupply": circulatingsupplies}
    
    outdf = pd.DataFrame(crypto_dict, index = currnames)
    del outdf['Names']
    
    return outdf

# Alternative Function for Pulling all Coin/Token Caps

In [17]:
def pullfulldf():
    site = 'https://coinmarketcap.com/all/views/all/'
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")

    table_start = '<table class="table js-summary-table dataTable no-footer" id="currencies-all" style="font-size: 14px; width: 100%;" role="grid">'
    table_end = '</table>'

    table = soup.find_all('table')[0]

    symbol = []
    cname = []
    mcapsusd = []
    pricesusd = []
    pricesbtc = []
    volumes24husd = []
    totalsupplies = []
    availablesupplies = []
    maxsupplies = []
    change1h = []
    change24h = []
    change7d = []
    i = 1
    for row in table.find_all('tr'):
        columns = str(row).split('\n')
        if not('class' in columns[0]):
            continue

        cmcname = columns[0].split('"id-')[1].replace('">','')

        apisite = 'https://api.coinmarketcap.com/v1/ticker/' + cmcname + '/'
        response = requests.get(apisite)
        json_data = json.loads(response.text)

        cname = cname + [cmcname]
        symbol = symbol + [json_data[0]['symbol']]
        mcapsusd = mcapsusd + [json_data[0]['market_cap_usd']]
        pricesusd = pricesusd + [json_data[0]['price_usd']]
        pricesbtc = pricesbtc + [json_data[0]['price_btc']]
        volumes24husd = volumes24husd + [json_data[0]['24h_volume_usd']]
        totalsupplies = totalsupplies + [json_data[0]['total_supply']]
        availablesupplies = availablesupplies + [json_data[0]['available_supply']]
        maxsupplies = maxsupplies + [json_data[0]['max_supply']]
        change1h = change1h + [json_data[0]['percent_change_1h']]
        change24h = change24h + [json_data[0]['percent_change_24h']]
        change7d = change7d + [json_data[0]['percent_change_7d']]
        
        print('Number: ' + str(i) + ' Coin: ' + json_data[0]['symbol'])
        sleep(1)
        i += 1

    fulldict = {'CMCname': cname,
                'Symbol': symbol,
                'Mcap': mcapsusd,
                'USDPrice': pricesusd,
                'BTCPrice': pricesbtc,           
                'Vol24h': volumes24husd,           
                'TotalSupply': totalsupplies,           
                'AvailableSupply': availablesupplies,    
                'MaxSupply': maxsupplies,
                'Change1h': change1h,
                'Change24h': change24h,
                'Change7d': change7d}
    fulldf = pd.DataFrame(fulldict,index = cname)
    
    return fulldf


# Creating dataframe for initial filtering
- Keep only marketcaps with less than 100M
- Remove tokens with 24 hour volumes less than 2% of marketcaps
- Remove tokens with circulating supply/total supply with less than 20%

In [10]:
# Creating DF for Coins

def microfilter1(mcapthreshold, volthreshold, supplythreshold):
    pagenmbr = 1
    coinlist = []
    while type(coindfpull(pagenmbr)) != str:
        coinlist = coinlist + [coindfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    coincirclist = []
    while type(coincircdf(pagenmbr)) != str:
        coincirclist = coincirclist + [coincircdf(pagenmbr)]
        pagenmbr += 1

    coinscirc = pd.concat(coincirclist, axis = 0)
    coins = pd.concat(coinlist, axis = 0)
    coins = coins[coins['MktCapUSD'] != '?']
    coins = pd.concat([coins.drop_duplicates(),coinscirc.drop_duplicates()], axis = 1, join = 'inner')

    # Creating DF for Tokens

    pagenmbr = 1
    tokenlist = []
    while type(tokendfpull(pagenmbr)) != str:
        tokenlist = tokenlist + [tokendfpull(pagenmbr)]
        pagenmbr += 1

    pagenmbr = 1
    tokencirclist = []
    while type(tokencirdf(pagenmbr)) != str:
        tokencirclist = tokencirclist + [tokencirdf(pagenmbr)]
        pagenmbr += 1

    tokenscirc = pd.concat(tokencirclist, axis = 0)
    tokens = pd.concat(tokenlist, axis = 0)
    tokens = tokens[tokens['MktCapUSD'] != '?']
    tokens = pd.concat([tokens.drop_duplicates(),tokenscirc.drop_duplicates()], axis = 1, join = 'inner')

    totaldf = pd.concat([coins,tokens], axis = 0)
    totaldf = totaldf[totaldf['CirculatingSupply'] != 'None']
    totaldf = totaldf[totaldf['Vol24USD'] != 'None']

    totaldf['MktCapUSD'] = pd.to_numeric(totaldf['MktCapUSD'])
    totaldf['CirculatingSupply'] = pd.to_numeric(totaldf['CirculatingSupply'])
    totaldf['TotalSupply'] = pd.to_numeric(totaldf['TotalSupply'])
    totaldf['Vol24USD'] = pd.to_numeric(totaldf['Vol24USD'])
    totaldf['VolRatio'] = totaldf['Vol24USD']/totaldf['MktCapUSD']
    totaldf['SupplyRatio'] = totaldf['CirculatingSupply']/totaldf['TotalSupply']
    
    totaldf = totaldf[totaldf['MktCapUSD'] <= mcapthreshold]
    totaldf = totaldf[totaldf['VolRatio'] >= volthreshold]
    totaldf = totaldf[totaldf['SupplyRatio'] >= supplythreshold]
    
    
    return totaldf

In [11]:
def checkannouncement(cmcname):

    site = 'https://coinmarketcap.com/currencies/' + cmcname + '/'

    res = requests.get(site)
    soup = str(bs(res.text, 'lxml'))

    if "Announcement" in soup:
        announcementsite = soup.split('<li><span class="glyphicon glyphicon-bullhorn text-gray" title="Announcement">')[1].split('<a href="')[1].split('" target')[0]
    else:
        announcementsite = "NA"

    if 'bitcointalk.org' in announcementsite:
        return 1, announcementsite
    else:
        return 0, 'NA'
    
def bctcheck(cmcname):

    annsite = checkannouncement(cmcname)[1]
    parsesoup = annsite.split('=')[0] + '=' + annsite.split('=')[1].split('.')[0]
    res = requests.get(parsesoup)
    soup = str(bs(res.text, 'lxml'))
    discussion = soup.split('<td class="middletext"')[1].split('<span class="prevnext">')[0]
    discussion = discussion.split('<div class="nav"')[0]
    if 'All' in discussion.split(parsesoup)[-1]:
        lastdiscuss = discussion.split(parsesoup)[-2]
    else:
        lastdiscuss = discussion.split(parsesoup)[-1]
        
    if '<td align="right" style="padding-right:' in discussion:
        lastpgIndex = lastdiscuss.split('">')[0]
    else:
        lastpgIndex = lastdiscuss.split('">')[0]

    lastpg = parsesoup + lastpgIndex
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    totpages = soup.split('<td class="middletext"')[1].split('[<b>')[1].split('</b>]')[0]

    forumparse = lastpg.split('=')[1]
    res = requests.get(lastpg)
    soup = str(bs(res.text, 'lxml'))
    lastmsg = soup.split(forumparse)[1].split('<tbody>')[-1].split('<div class="smalltext">')[-1].split('</div>')[0]
    
    return totpages, lastmsg

In [ ]:
'''
# Using old method to build dataframe, no change fields available

cmcdf = microfilter1(100000000, 0.02, 0.1)

names = list(cmcdf.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        totpgs, last = bctcheck(cmcname)
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcdf,bctdf],axis = 1)
filterdf.to_csv('cmc2.csv')

'''

In [25]:
cmcfull = pullfulldf()

names = list(cmcfull.index)
announcements = []
bctpages = []
lastentries = []

for cmcname in names:
    bctentry, annsite = checkannouncement(cmcname)
    announcements = announcements + [bctentry]
    if bctentry == 1:
        try:
            totpgs, last = bctcheck(cmcname)
        except:
            totpgs, last = (0,0)
        
    else:
        totpgs, last = (0,0)
        
    bctpages = bctpages + [totpgs]
    lastentries = lastentries + [last]
    print('Name: ' + str(len(bctpages)) + ' Alt: ' + cmcname)
    sleep(1)
    
bctdf = pd.DataFrame({'bctpages': bctpages,
                     'lastentry': lastentries},
                    index = names)
filterdf = pd.concat([cmcfull,bctdf],axis = 1)

filterdf.to_csv('cmc.csv')

Number: 1 Coin: BTC
Number: 2 Coin: ETH
Number: 3 Coin: XRP
Number: 4 Coin: BCH
Number: 5 Coin: ADA
Number: 6 Coin: XEM
Number: 7 Coin: LTC
Number: 8 Coin: XLM
Number: 9 Coin: TRX
Number: 10 Coin: MIOTA
Number: 11 Coin: DASH
Number: 12 Coin: NEO
Number: 13 Coin: XMR
Number: 14 Coin: EOS
Number: 15 Coin: QTUM
Number: 16 Coin: ICX
Number: 17 Coin: BTG
Number: 18 Coin: ETC
Number: 19 Coin: LSK
Number: 20 Coin: XRB
Number: 21 Coin: SC
Number: 22 Coin: OMG
Number: 23 Coin: BCN
Number: 24 Coin: XVG
Number: 25 Coin: BCC
Number: 26 Coin: DCN
Number: 27 Coin: ZEC
Number: 28 Coin: BTS
Number: 29 Coin: PPT
Number: 30 Coin: DOGE
Number: 31 Coin: BNB
Number: 32 Coin: STRAT
Number: 33 Coin: KCS
Number: 34 Coin: SNT
Number: 35 Coin: ARDR
Number: 36 Coin: STEEM
Number: 37 Coin: USDT
Number: 38 Coin: WAVES
Number: 39 Coin: VEN
Number: 40 Coin: DGB
Number: 41 Coin: DRGN
Number: 42 Coin: KMD
Number: 43 Coin: HSR
Number: 44 Coin: VERI
Number: 45 Coin: GNT
Number: 46 Coin: KIN
Number: 47 Coin: ETN
Number: 

Number: 372 Coin: BET
Number: 373 Coin: CAT
Number: 374 Coin: BITUSD
Number: 375 Coin: BBR
Number: 376 Coin: XPM
Number: 377 Coin: FRST
Number: 378 Coin: OBITS
Number: 379 Coin: SEQ
Number: 380 Coin: XVC
Number: 381 Coin: CSNO
Number: 382 Coin: RKC
Number: 383 Coin: XMCC
Number: 384 Coin: BCY
Number: 385 Coin: TFL
Number: 386 Coin: CANN
Number: 387 Coin: SPR
Number: 388 Coin: SUMO
Number: 389 Coin: DBIX
Number: 390 Coin: TX
Number: 391 Coin: OPT
Number: 392 Coin: WISH
Number: 393 Coin: BTM
Number: 394 Coin: DOPE
Number: 395 Coin: XUC
Number: 396 Coin: OXY
Number: 397 Coin: LEO
Number: 398 Coin: BQ
Number: 399 Coin: SNRG
Number: 400 Coin: PFR
Number: 401 Coin: DICE
Number: 402 Coin: PRIX
Number: 403 Coin: UNO
Number: 404 Coin: UFR
Number: 405 Coin: PLU
Number: 406 Coin: SPHR
Number: 407 Coin: 1337
Number: 408 Coin: SYNX
Number: 409 Coin: DOT
Number: 410 Coin: ATMS
Number: 411 Coin: NOBL
Number: 412 Coin: INXT
Number: 413 Coin: HYP
Number: 414 Coin: BWK
Number: 415 Coin: ZEIT
Number: 416

Number: 738 Coin: CORG
Number: 739 Coin: RBT
Number: 740 Coin: PCOIN
Number: 741 Coin: KUSH
Number: 742 Coin: ETG
Number: 743 Coin: CCO
Number: 744 Coin: BCF
Number: 745 Coin: EGAS
Number: 746 Coin: LCP
Number: 747 Coin: DDF
Number: 748 Coin: ALTCOM
Number: 749 Coin: XRA
Number: 750 Coin: BIGUP
Number: 751 Coin: AMMO
Number: 752 Coin: FNC
Number: 753 Coin: GRT
Number: 754 Coin: XCT
Number: 755 Coin: VLT
Number: 756 Coin: RLT
Number: 757 Coin: CUBE
Number: 758 Coin: EMD
Number: 759 Coin: ITI
Number: 760 Coin: NTWK
Number: 761 Coin: UNY
Number: 762 Coin: MONK
Number: 763 Coin: STARS
Number: 764 Coin: BLC
Number: 765 Coin: GRIM
Number: 766 Coin: XGR
Number: 767 Coin: MARS
Number: 768 Coin: GAP
Number: 769 Coin: RBX
Number: 770 Coin: GAIA
Number: 771 Coin: UNIC
Number: 772 Coin: BOLI
Number: 773 Coin: CCRB
Number: 774 Coin: 888
Number: 775 Coin: LTB
Number: 776 Coin: CHESS
Number: 777 Coin: BTSR
Number: 778 Coin: MAD
Number: 779 Coin: CAT
Number: 780 Coin: BITBTC
Number: 781 Coin: MOJO
Num

Number: 1096 Coin: TSL
Number: 1097 Coin: AMM
Number: 1098 Coin: HTML
Number: 1099 Coin: CAT
Number: 1100 Coin: PRO
Number: 1101 Coin: QBT
Number: 1102 Coin: ACE
Number: 1103 Coin: HPY
Number: 1104 Coin: BCX
Number: 1105 Coin: FRGC
Number: 1106 Coin: CAPP
Number: 1107 Coin: SBTC
Number: 1108 Coin: NGC
Number: 1109 Coin: SHND
Number: 1110 Coin: INF
Number: 1111 Coin: B2X
Number: 1112 Coin: ENT
Number: 1113 Coin: DIM
Number: 1114 Coin: DEW
Number: 1115 Coin: LLT
Number: 1116 Coin: CMS
Number: 1117 Coin: MKR
Number: 1118 Coin: VIU
Number: 1119 Coin: WC
Number: 1120 Coin: UGT
Number: 1121 Coin: MDS
Number: 1122 Coin: CLUB
Number: 1123 Coin: IRL
Number: 1124 Coin: XTZ
Number: 1125 Coin: AI
Number: 1126 Coin: CMS
Number: 1127 Coin: FIL
Number: 1128 Coin: BIG
Number: 1129 Coin: HBC
Number: 1130 Coin: HTML5
Number: 1131 Coin: BTCA
Number: 1132 Coin: IGNIS
Number: 1133 Coin: IFC
Number: 1134 Coin: XID
Number: 1135 Coin: BCDN
Number: 1136 Coin: PAYP
Number: 1137 Coin: LBTC
Number: 1138 Coin: ESC

Name: 63 Alt: nexus
Name: 64 Alt: qash
Name: 65 Alt: aeternity
Name: 66 Alt: power-ledger
Name: 67 Alt: request-network
Name: 68 Alt: aion
Name: 69 Alt: wax
Name: 70 Alt: substratum
Name: 71 Alt: bytom
Name: 72 Alt: enigma-project
Name: 73 Alt: rchain
Name: 74 Alt: digitalnote
Name: 75 Alt: nxt
Name: 76 Alt: maidsafecoin
Name: 77 Alt: quantstamp
Name: 78 Alt: byteball
Name: 79 Alt: gas
Name: 80 Alt: santiment
Name: 81 Alt: monacoin
Name: 82 Alt: iconomi
Name: 83 Alt: chainlink
Name: 84 Alt: syscoin
Name: 85 Alt: bitcoindark
Name: 86 Alt: zcoin
Name: 87 Alt: poet
Name: 88 Alt: blockv
Name: 89 Alt: walton
Name: 90 Alt: digixdao
Name: 91 Alt: tenx
Name: 92 Alt: paccoin
Name: 93 Alt: time-new-bank
Name: 94 Alt: neblio
Name: 95 Alt: raiden-network-token
Name: 96 Alt: gnosis-gno
Name: 97 Alt: deepbrain-chain
Name: 98 Alt: civic
Name: 99 Alt: bancor
Name: 100 Alt: ethlend
Name: 101 Alt: zclassic
Name: 102 Alt: achain
Name: 103 Alt: gxshares
Name: 104 Alt: gamecredits
Name: 105 Alt: storm
Name

Name: 403 Alt: unobtanium
Name: 404 Alt: upfiring
Name: 405 Alt: pluton
Name: 406 Alt: sphere
Name: 407 Alt: 1337coin
Name: 408 Alt: syndicate
Name: 409 Alt: dotcoin
Name: 410 Alt: atmos
Name: 411 Alt: noblecoin
Name: 412 Alt: internxt
Name: 413 Alt: hyperstake
Name: 414 Alt: bulwark
Name: 415 Alt: zeitcoin
Name: 416 Alt: alqo
Name: 417 Alt: pesetacoin
Name: 418 Alt: 2give
Name: 419 Alt: carboncoin
Name: 420 Alt: encryptotel
Name: 421 Alt: dcorp
Name: 422 Alt: neutron
Name: 423 Alt: social-send
Name: 424 Alt: goldcoin
Name: 425 Alt: breakout
Name: 426 Alt: nvo
Name: 427 Alt: xaurum
Name: 428 Alt: crowdcoin
Name: 429 Alt: dnotes
Name: 430 Alt: auroracoin
Name: 431 Alt: memetic
Name: 432 Alt: luxcoin
Name: 433 Alt: the-champcoin
Name: 434 Alt: gobyte
Name: 435 Alt: belacoin
Name: 436 Alt: bitzeny
Name: 437 Alt: veriumreserve
Name: 438 Alt: vsync-vsx
Name: 439 Alt: quark
Name: 440 Alt: creditbit
Name: 441 Alt: breakout-stake
Name: 442 Alt: universal-currency
Name: 443 Alt: real
Name: 444 

Name: 736 Alt: dix-asset
Name: 737 Alt: hicoin
Name: 738 Alt: corgicoin
Name: 739 Alt: rimbit
Name: 740 Alt: pioneer-coin
Name: 741 Alt: kushcoin
Name: 742 Alt: ethereum-gold
Name: 743 Alt: ccore
Name: 744 Alt: bitcoinfast
Name: 745 Alt: ethgas
Name: 746 Alt: litecoin-plus
Name: 747 Alt: digital-developers-fund
Name: 748 Alt: altcommunity-coin
Name: 749 Alt: ratecoin
Name: 750 Alt: bigup
Name: 751 Alt: ammo-rewards
Name: 752 Alt: fincoin
Name: 753 Alt: grantcoin
Name: 754 Alt: c-bit
Name: 755 Alt: veltor
Name: 756 Alt: roulettetoken
Name: 757 Alt: digicube
Name: 758 Alt: emerald
Name: 759 Alt: iticoin
Name: 760 Alt: network-token
Name: 761 Alt: unity-ingot
Name: 762 Alt: monkey-project
Name: 763 Alt: starcash-network
Name: 764 Alt: blakecoin
Name: 765 Alt: grimcoin
Name: 766 Alt: goldreserve
Name: 767 Alt: marscoin
Name: 768 Alt: gapcoin
Name: 769 Alt: ripto-bux
Name: 770 Alt: gaia
Name: 771 Alt: unicoin
Name: 772 Alt: bolivarcoin
Name: 773 Alt: cryptocarbon
Name: 774 Alt: octocoin
Nam

Name: 1058 Alt: genstake
Name: 1059 Alt: dappster
Name: 1060 Alt: orlycoin
Name: 1061 Alt: cashcoin
Name: 1062 Alt: morningstar-payments
Name: 1063 Alt: mtmgaming
Name: 1064 Alt: impulsecoin
Name: 1065 Alt: bowscoin
Name: 1066 Alt: tagrcoin
Name: 1067 Alt: crtcoin
Name: 1068 Alt: letitride
Name: 1069 Alt: osmiumcoin
Name: 1070 Alt: ponzicoin
Name: 1071 Alt: cryptoescudo
Name: 1072 Alt: xonecoin
Name: 1073 Alt: sydpak
Name: 1074 Alt: frazcoin
Name: 1075 Alt: gbcgoldcoin
Name: 1076 Alt: ccminer
Name: 1077 Alt: lex4all
Name: 1078 Alt: biobar
Name: 1079 Alt: concoin
Name: 1080 Alt: sojourn
Name: 1081 Alt: ulatech
Name: 1082 Alt: pizzacoin
Name: 1083 Alt: burstocean
Name: 1084 Alt: caliphcoin
Name: 1085 Alt: applecoin-apw
Name: 1086 Alt: atmcoin
Name: 1087 Alt: nebulas-token
Name: 1088 Alt: smartmesh
Name: 1089 Alt: genaro-network
Name: 1090 Alt: bibox-token
Name: 1091 Alt: game
Name: 1092 Alt: bitcoin-diamond
Name: 1093 Alt: olympus-labs
Name: 1094 Alt: qlink
Name: 1095 Alt: sophiatx
Name:

Name: 1375 Alt: royalties
Name: 1376 Alt: embercoin
Name: 1377 Alt: casinocoin
Name: 1378 Alt: ocow
Name: 1379 Alt: bitcoin-unlimited
Name: 1380 Alt: xylo
Name: 1381 Alt: santa-coin
Name: 1382 Alt: qcash
Name: 1383 Alt: farstcoin
Name: 1384 Alt: madcoin
